In [1]:
#load the modules
%load_ext autoreload 
%autoreload 2

In [2]:
#cd '../'

/Users/nikhilsingh/Downloads/Dalmia_machine_learning/Nikhil


In [3]:
import numpy as np 
import pandas as pd 
from utility.db_data import connection_db
import psycopg2
from utility.datetime_util import create_date_columns

In [4]:
host="database-dalmia.cms6cet2hwec.ap-south-1.rds.amazonaws.com"
database="dalmia_digital"
user="postgres"
password="dalmia123"
sql_query = "SELECT * FROM dalmia_sales.sales_data where billing_date > '2021-04-01' and sold_to_party_code between 7000000 and 7099999"

In [5]:
df = connection_db(host=host, database=database, user=user, password=password , sql_query=sql_query)

/Users/nikhilsingh/Downloads/Dalmia_machine_learning/Nikhil/utility/db_data.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


In [6]:
df.head()

,month,billing_date,sales_region,sales_zone,sales_type,manufacturing_plant,manufacturing_plant_name,plant,plant_name,sold_to_party_code,...,direct_depot,incoterms_inco,mode_of_transport,t_zone,tzone_desc,brand,product,bill_quantity,gross_bill_price,zbill_invoice_bill
0,April-2023,2023-04-29,East,WBS,Trade,1322,DCBL FACTORY-OBCW,C050,"DCBL,Naihati",7001674,...,Depot,FTP,Road,7412350905,KALYANI,KONARK,PCC,6.0,41160,40680.0
1,April-2023,2023-04-29,East,WBS,Trade,1322,DCBL FACTORY-OBCW,C050,"DCBL,Naihati",7001674,...,Depot,FTP,Road,7412350909,KALYANI UNIVERSITY,DSP,DSP,12.0,95520,94560.0
2,April-2023,2023-04-29,East,WBS,Trade,1322,DCBL FACTORY-OBCW,C050,"DCBL,Naihati",7001909,...,Depot,FTP,Road,7431661201,GORIFA,KONARK,PCC,5.0,34300,33900.0
3,April-2023,2023-04-29,East,WBS,Trade,1322,DCBL FACTORY-OBCW,C050,"DCBL,Naihati",7001909,...,Depot,FTP,Road,7431261206,PADMALAVPUR,DSP,DSP,10.5,83580,82740.0
4,April-2023,2023-04-29,East,WBS,Trade,1322,DCBL FACTORY-OBCW,C050,"DCBL,Naihati",7001909,...,Depot,FTP,Road,7431351202,JETIA,DALMIA,PCC,5.0,36300,35900.0


In [7]:
imp_cols = ['month', 'billing_date','sales_region','sold_to_party_code','sold_to_party_district_name','ship_to_party_code',
            'ship_to_party_district','brand', 'product', 'bill_quantity']
df_sales = df[imp_cols].copy()

In [8]:
df_sales.head()

,month,billing_date,sales_region,sold_to_party_code,sold_to_party_district_name,ship_to_party_code,ship_to_party_district,brand,product,bill_quantity
0,April-2023,2023-04-29,East,7001674,NADIA,77011143,NADIA,KONARK,PCC,6.0
1,April-2023,2023-04-29,East,7001674,NADIA,77009946,NADIA,DSP,DSP,12.0
2,April-2023,2023-04-29,East,7001909,24 PRAGANA NORTH,77089042,24 PRAGANA NORTH,KONARK,PCC,5.0
3,April-2023,2023-04-29,East,7001909,24 PRAGANA NORTH,77092521,24 PRAGANA NORTH,DSP,DSP,10.5
4,April-2023,2023-04-29,East,7001909,24 PRAGANA NORTH,77053161,24 PRAGANA NORTH,DALMIA,PCC,5.0


In [44]:
#df_sales[df_sales['sold_to_party_code']==df_sales['ship_to_party_code']].to_csv('data/dealer_subdealer.csv')

In [9]:
df_sales = create_date_columns([df_sales])
df_group_distt = df_sales[['Month_Year',
                        'bill_quantity','sold_to_party_district_name']].groupby(['sold_to_party_district_name' , 'Month_Year']).agg({'bill_quantity': ['sum']}).reset_index()

In [67]:
df_group_distt.columns = ['sold_to_party_district_name', 'month' , 'Sum']
season_pivot = pd.pivot_table(df_group_distt, values='Sum',columns='month', index = 'sold_to_party_district_name')
season_pivot

month,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05
sold_to_party_district_name,,,,,,,,,,,,,,,,,,,,,
24 PRAGANA NORTH,14482.90,9103.00,8518.85,8349.95,8355.7,7395.80,8340.45,7803.65,12212.75,11586.50,...,9258.60,9742.60,7850.65,9578.85,11246.10,9421.90,9802.65,7860.25,4280.40,4256.85
24 PRAGANA SOUTH,20494.75,10664.85,10153.05,9379.25,9059.2,7297.80,9178.75,10440.35,17104.05,17403.25,...,10525.80,10100.45,9790.65,11885.60,12820.20,11608.40,13698.95,20425.15,9574.55,8849.50
ALIPURDUAR,3339.90,3573.35,3685.35,2332.05,2132.5,2427.00,2819.30,4401.90,7555.90,6114.00,...,4981.25,4726.95,2778.80,3732.80,3439.90,4899.85,4003.65,6106.00,4381.65,4341.90
ANGUL,6942.25,5449.30,6433.35,5603.65,6982.9,6497.95,7571.55,6692.10,8560.25,8405.05,...,4314.60,6171.90,5480.35,6183.55,7375.55,7342.95,7711.80,10152.80,4827.20,4425.90
ANUPPUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Tripura,5085.65,3139.90,2598.05,3269.05,3534.3,3728.55,3952.05,3894.20,5334.20,5768.40,...,5007.90,4466.50,3461.05,5162.95,5831.25,6276.90,6478.00,7684.85,6178.30,3982.15
Wokha,560.00,555.00,280.00,190.00,214.0,100.00,175.00,100.00,150.00,175.00,...,175.00,125.00,150.00,150.00,60.00,125.00,75.00,150.00,150.00,75.00
YADADRI BHUVANAGIRI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,120.00,120.00,108.00,95.50,126.50,132.55,161.00,124.00,100.50,95.00


In [68]:
df_group = df_sales[['Month_Year',
                        'bill_quantity','sold_to_party_code']].groupby(['sold_to_party_code' , 'Month_Year']).agg({'bill_quantity': ['sum']}).reset_index()
df_group.columns = ['sold_to_party_code', 'month' , 'Sum']
pivot = pd.pivot_table(df_group, values='Sum',columns='month', index = 'sold_to_party_code')



In [69]:
pivot = pivot.fillna(0)
pivot

month,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05
sold_to_party_code,,,,,,,,,,,,,,,,,,,,,
7000004,255.5,85.0,207.0,102.0,73.5,74.5,43.0,25.0,76.0,53.0,...,20.0,20.0,20.0,20.0,122.5,75.0,90.0,155.0,46.0,86.0
7000010,57.0,37.0,0.0,24.0,30.0,0.0,41.0,24.0,25.0,24.0,...,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0
7000015,4.0,50.0,63.0,62.0,91.0,68.5,70.5,77.0,50.0,44.5,...,54.5,45.0,74.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0
7000019,12.0,50.0,133.0,104.0,165.0,123.5,0.0,7.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7000022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,24.0,22.0,5.0,21.0,24.0,0.0,29.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7040778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
7040779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
7040781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.5


In [70]:
pivot.columns

Index(['2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09',
       '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03',
       '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
       '2023-04', '2023-05'],
      dtype='object', name='month')

In [71]:
df_sales_1 = df_sales[['sold_to_party_code','sold_to_party_district_name']]

In [72]:
#df_sales_1.index = df_sales_1['sold_to_party_code']
#df_sales_1.drop('sold_to_party_code' , axis =1 , inplace = True)

In [73]:
df_sales_1 = df_sales_1[['sold_to_party_code', 'sold_to_party_district_name']].drop_duplicates()

In [74]:
df_sales_1 = pd.DataFrame(df_sales_1)

In [75]:
df_sales_1.index = df_sales_1['sold_to_party_code']
df_sales_1.drop('sold_to_party_code', axis = 1, inplace = True)

In [76]:
result_matrix = pd.merge(pivot , df_sales_1 , left_index=True, right_index=True , how = 'inner')

In [106]:
nov_df = result_matrix[result_matrix['2022-11']>0]

In [107]:
dist_count = pd.DataFrame()
dist_count = oct_df[['2022-11', 
                                'sold_to_party_district_name']].groupby(['sold_to_party_district_name']).agg({'2022-11':['count']}).reset_index()







In [108]:
dist_count.columns = ['sold_to_party_district_name','count']

In [109]:
dist_count

,sold_to_party_district_name,count
0,24 PRAGANA NORTH,45
1,24 PRAGANA SOUTH,39
2,ALIPURDUAR,12
3,ANGUL,45
4,ARARIA,9
...,...,...
350,West Tripura,34
351,Wokha,1
352,YADADRI BHUVANAGIRI,2
353,YADGIR,3


In [110]:
#dist_count = dist_count[dist_count['2022-10']'Active']
dist_count.index = dist_count['sold_to_party_district_name']
dist_count.drop('sold_to_party_district_name', axis = 1 , inplace = True)

In [113]:
df_test= pd.merge(season_pivot['2022-11'] , dist_count['count'] , left_index=True , right_index=True , how = 'left') 

In [114]:
df_test = df_test.fillna(0)

In [117]:
df_test['mean_bemchmark'] = df_test['2022-11']/df_test['count']
df_test

,2022-11,count,mean_Q-2,mean_bemchmark
sold_to_party_district_name,,,,
24 PRAGANA NORTH,9578.85,45.0,212.863333,212.863333
24 PRAGANA SOUTH,11885.60,39.0,304.758974,304.758974
ALIPURDUAR,3732.80,12.0,311.066667,311.066667
ANGUL,6183.55,45.0,137.412222,137.412222
ANUPPUR,0.00,0.0,NaN,NaN
...,...,...,...,...
West Tripura,5162.95,34.0,151.851471,151.851471
Wokha,150.00,1.0,150.000000,150.000000
YADADRI BHUVANAGIRI,95.50,2.0,47.750000,47.750000


ValueError: left_index parameter must be of type bool, not <class 'str'>

In [103]:
df_sal

,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,sold_to_party_district_name
sold_to_party_code,,,,,,,,,,,,,,,,,,,,,
7000004,255.5,85.0,207.0,102.0,73.5,74.5,43.0,25.0,76.0,53.0,...,20.0,20.0,20.0,122.5,75.0,90.0,155.0,46.0,86.0,BHADRAK
7000010,57.0,37.0,0.0,24.0,30.0,0.0,41.0,24.0,25.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,CUTTACK
7000015,4.0,50.0,63.0,62.0,91.0,68.5,70.5,77.0,50.0,44.5,...,45.0,74.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,PURI
7000019,12.0,50.0,133.0,104.0,165.0,123.5,0.0,7.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PURI
7000022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24.0,22.0,5.0,21.0,24.0,0.0,29.0,0.0,10.0,KHURDA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7040778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,CHATRA
7040779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,MUZAFFARPUR
7040781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.5,SAHEBGANJ
